# **Web Scrapping con BeautifulSoup**

In [1]:
# Imports
from bs4 import BeautifulSoup
import requests

In [2]:
# Versiones
import bs4 # Solo para chequeo
print(f'Versión BeautifulSoup: {bs4.__version__}')
print(f'Versión Requests: {requests.__version__}')

Versión BeautifulSoup: 4.11.1
Versión Requests: 2.31.0


In [14]:
# Empezamos el Scrapping
# 1. Obtener el enlace
URL_BASE = 'https://scrapepark.org/'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

# 2. "Parsear" HTML
soup = BeautifulSoup(html_obtenido, "html.parser")

### **El método `.find()`**

Nos permite quedarnos con la información asociada a una etiqueta de HTML

In [4]:
primer_h2 = soup.find('h2')
primer_h2

<h2>¿Por qué comprar con nosotros?</h2>

In [5]:
# Solo el texto
primer_h2.text

'¿Por qué comprar con nosotros?'

### **El método `.find_all()`**

Busca **TODOS** los elementos de la página con esa etiqueta y devuelve una "lista" que los contiene (en realidad devuelve un objeto de la clase *bs4.element.ResultSet*).

In [6]:
h2_todos = soup.find_all('h2')
h2_todos

[<h2>¿Por qué comprar con nosotros?</h2>,
 <h2>
                   #<span>Novedades</span>
 </h2>,
 <h2>Nuestros productos</h2>,
 <h2>Testimonios de clientes</h2>,
 <h2 class="heading-container">Precios</h2>]

In [7]:
# ARGUMENTOS
# Si usamos el parámetro limit=1, emulamos al método .find()
h2_uno_solo = soup.find_all('h2', limit=1)
h2_uno_solo

[<h2>¿Por qué comprar con nosotros?</h2>]

In [8]:
# Podemos iterar sobre el objeto
for seccion in h2_todos:
    print(seccion.text)

¿Por qué comprar con nosotros?

                  #Novedades

Nuestros productos
Testimonios de clientes
Precios


In [9]:
# get_text() para mas funcionalidades
for seccion in h2_todos:
    print(seccion.get_text(strip=True))

¿Por qué comprar con nosotros?
#Novedades
Nuestros productos
Testimonios de clientes
Precios


### **Utilizando atributos de las etiquetas**

In [10]:
# Clase
divs = soup.find_all('div', class_ = "heading-container heading-center")

for div in divs:
    print(div)
    print(" ")

<div class="heading-container heading-center" id="about">
<h2>¿Por qué comprar con nosotros?</h2>
</div>
 
<div class="heading-container heading-center" id="products">
<h2>Nuestros productos</h2>
</div>
 
<div class="heading-container heading-center">
<h3>Suscríbete para obtener descuentos y ofertas</h3>
</div>
 
<div class="heading-container heading-center">
<h2>Testimonios de clientes</h2>
</div>
 


In [11]:
# Todas las etiquetas que tengan el artibuto "src"
src_todos = soup.find_all(src=True)

for elemento in src_todos:
    if elemento['src'].endswith(".jpg"):
        print(elemento)

<img alt="Parque de patinaje" src="../images/slider-bg.jpg"/>
<img alt="Patineta" src="../images/p2.jpg"/>


### **Descargando contenido**

In [ ]:
# ¡¡¡CORRER ESTA CELDA DESCARGA TODAS LAS IMAGENES DEL SITIO!!!
# Ejercicio: Descargar todas las imágenes
url_imagenes = []

for i, img in enumerate(src_todos):

    if img['src'].endswith(".png"):

        print(img['src'])
        r = requests.get(f"{URL_BASE}{img['src']}")

        with open(f'img_{i}.png', 'wb') as f:
            f.write(r.content)

### **Etiquetas iframe y table**

In [17]:
# Información de tablas
URL_TABLA = soup.find_all('iframe')[0]['src']

request_tabla = requests.get(f'{URL_BASE}{URL_TABLA}')

html_tabla = request_tabla.text
soup_tabla = BeautifulSoup(html_tabla, 'html.parser')
soup_tabla.find('table')

#Los modelos faltantes se muestran en color rojo en la tabla
productos_faltantes = soup_tabla.find_all(['th', 'td'], attrs={'style': 'color: red;'})
productos_faltantes = [talle.text for talle in productos_faltantes]

print(productos_faltantes)

['Longboard', '$80', '$85', '$90', '$62', '$150']


In [20]:
divs = soup.find_all('div', class_='detail-box')
productos = []
precios = []

for div in divs:
    if (div.h6 is not None) and ('Skateboard' in div.h5.text):
        producto = div.h5.get_text(strip=True)
        precio = div.h6.get_text(strip=True).replace('$', '')
        # Se pueden agregar filtros
        print(f'producto: {producto:<16} | precio: {precio}')
        productos.append(producto)
        precios.append(precio)

producto: New Skateboard1  | precio: 75
producto: Used Skateboard2 | precio: 80
producto: New Skateboard3  | precio: 68
producto: Used Skateboard4 | precio: 70
producto: New Skateboard5  | precio: 75
producto: New Skateboard6  | precio: 58
producto: New Skateboard7  | precio: 80
producto: New Skateboard8  | precio: 35
producto: New Skateboard9  | precio: 165
producto: Used Skateboard10 | precio: 54
producto: Used Skateboard11 | precio: 99
producto: New Skateboard12 | precio: 110


### **Cambios que dependen de la URL**

In [23]:
URL_BASE = 'https://scrapepark.org/contact'

for i in range(1, 3):
    URL_FINAL = f"{URL_BASE}{i}"
    print(URL_FINAL)
    r = requests.get(URL_FINAL)
    soup = BeautifulSoup(r.text, "html.parser")
    print(soup.h5.text)

https://scrapepark.org/contact1
Text that changes between pages in contact 1 :)
https://scrapepark.org/contact2
Text that changes between pages in contact 2 :)


### **Datos que no sabemos en qué parte de la página se encuentran**

In [24]:
# Expresiones regulares
import re

# 1. Obtener el HTML
URL_BASE = 'https://scrapepark.org'
pedido_obtenido = requests.get(URL_BASE)
html_obtenido = pedido_obtenido.text

# 2. "Parsear" el HTML
soup = BeautifulSoup(html_obtenido, "html.parser")

telefonos = soup.find_all(string=re.compile("\d+-\d+-\d+"))
telefonos

[': 4-444-4444']

In [27]:
emails = soup.find_all(string=re.compile("[a-zA-Z0-9]+(?:\.[a-zA-Z0-9]+)*@[a-zA-z0-9]+(?:\.[a-zA-Z0-9]+)*$"))
emails

['sales@mail.com']

### **Moviéndonos por el árbol**

In [33]:
copyrights = soup.find_all(string=re.compile("©"))
copyrights

['© 2022 ']

In [34]:
primer_copyright = copyrights[0]
primer_copyright.parent

<p>© 2022 <span>All Rights Reserved</span>.
        <a href="https://html.design/" rel="noopener noreferrer" target="_blank">Created with Free Html Templates</a>.
      </p>

In [36]:
menu = soup.find(string=re.compile("MENU"))
menu.parent

<h3 class="menu">MENU</h3>

In [37]:
menu.parent.find_next_siblings()

[<ul>
 <li><a href="#">Home</a></li>
 <li><a href="#">About</a></li>
 <li><a href="#">Services</a></li>
 <li><a href="#">Testimonials</a></li>
 <li><a href="#">Contact</a></li>
 </ul>]

### **Comentário sobre excepciones**

In [38]:
strings_a_buscar = ["MENU", "©", "Carpincho", "Skateboard"]

for string in strings_a_buscar:
    try:
        resultado = soup.find(string=re.compile(string))
        print(resultado.text)
    except AttributeError:
        print(f"El string '{string}' no fue encontrado.")

MENU
© 2022 
El string 'Carpincho' no fue encontrado.
New Skateboard


### **Almacenamiento de los datos**

In [39]:
import csv

productos.insert(0, "productos")
precios.insert(0, "precios")
datos = dict(zip(productos, precios))

with open('datos.csv', 'w') as f:
    w = csv.writer(f)
    w.writerows(datos.items())